<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/GroceryData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install faker

     |████████████████████████████████| 1.5 MB 4.7 MB/s 


In [68]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/GroceryPrices.xlsx -O GroceryPrices.xlsx

In [339]:
import requests
from lxml import html
import numpy as np
import pandas as pd
from collections import deque
from faker import Faker
import plotly.express as px

In [252]:
idx=pd.IndexSlice

In [64]:
rng=np.random.RandomState()

In [69]:
df_grocery = pd.read_excel('GroceryPrices.xlsx')

In [169]:
fake=Faker()
names=[fake.name_nonbinary() for i in range(100)]

In [315]:
now = pd.to_datetime('now').round('s')
df_names=[]
for _ in range(100):
    name = rng.choice(names)
    starts = now + pd.to_timedelta(rng.randint(100,24*3600*30,size=rng.poisson(3)+1),'s')
    df_start=[]
    for start in starts:
        n_items = rng.poisson(10)+1
        times = start + pd.to_timedelta(np.cumsum(rng.poisson(5,n_items))+1,'s')
        items = rng.choice(df_grocery['Item'],n_items)
        quantity = rng.poisson(0.5, size=n_items)+1
        df_start.append(pd.DataFrame(dict(time=times, item=items,quantity=quantity)))

    df_name=pd.concat(df_start)
    df_name['name']=name
    df_names.append(df_name)
df_noprices=pd.concat(df_names)

In [324]:
df_withPrice

,time,item,quantity,name,Price
0,2022-06-09 18:15:54,Yogurt,1,Heather Gillespie,1.5
1,2022-06-04 04:07:18,Yogurt,1,Heather Gillespie,1.5
2,2022-05-12 23:18:12,Yogurt,1,Heather Gillespie,1.5
3,2022-06-03 11:53:40,Yogurt,2,Michelle Brock,1.5
4,2022-06-10 18:41:13,Yogurt,1,Michelle Brock,1.5
...,...,...,...,...,...
4502,2022-05-13 18:47:31,Onions,1,Cassandra Mann,2.0
4503,2022-06-03 12:16:44,Onions,1,Jeffery Gomez,2.0
4504,2022-06-02 00:20:45,Onions,1,Jeffery Gomez,2.0
4505,2022-05-29 00:59:10,Onions,2,Mitchell White,2.0


In [325]:
df_withPrice=df_noprices.merge(df_grocery, left_on='item', right_on='Item').drop('Item',axis=1)
df_withPrice['subtotal']=df_withPrice['Price']*df_withPrice['quantity']

In [326]:
df_shopping=df_withPrice.sort_values(by=['name','time'])

In [293]:
def trip_count(d):
    d['trip']=np.cumsum(d['time'].diff()>pd.to_timedelta(60,'s'))
    return d 

In [327]:
df_trip = df_shopping.groupby('name').apply(trip_count)

In [384]:
def checkout_duration_total(d):
    return pd.Series(dict(duration=(pd.Timedelta(d['time'].max()-d['time'].min()))/pd.to_timedelta(1,'s'),
                          total=d['subtotal'].sum()))

In [385]:
df_checkout=df_trip.groupby(['name','trip']).apply(checkout_duration_total)

In [386]:
df_first16_names=df_checkout.loc[idx[:df_checkout.index.levels[0][15],:],:].reset_index()

In [388]:
fig=px.bar(df_first16_names,x='trip',y='duration',facet_col='name',facet_col_wrap=4, template='plotly_dark',height=800)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("name=","")))

In [389]:
fig2=px.bar(df_first16_names,x='trip',y='total',facet_col='name',facet_col_wrap=4, template='plotly_dark',height=800)
fig2.for_each_annotation(lambda a: a.update(text=a.text.replace("name=","")))